sources:
docs.streamlit.io/library/get-started

# Introduction

In [16]:
# Import Libraries
import numpy as np
import pandas as pd

pd.set_option("display.precision", 2)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
from pandas.api.types import (
    is_categorical_dtype,
    is_datetime64_any_dtype,
    is_numeric_dtype,
    is_object_dtype,
)

import plotly.figure_factory as ff

## Streamlit

In [17]:
import streamlit as st

In [18]:
st.set_page_config(
    page_title="Player Batting",
    page_icon="⚾",
    layout="wide",
    initial_sidebar_state="expanded",
    menu_items={
        'Report a bug': 'https://www.extremelycoolapp.com/help',
        'Get help': "https://www.extremelycoolapp.com/bug",
        'About': "# This is a header. This is an *extremely* cool app!"
    }
)

In [19]:
def add_logo():
    st.markdown(
        """
        <style>
            [data-testid="stSidebarNav"] {
                background-image: url(https://i.imgur.com/sLSMBYJ.png);
                background-repeat: no-repeat;
                padding-top: 60px;
                background-position: 20px 20px;
            }
            [data-testid="stSidebarNav"]::before {
                content: "Navigation Menu";
                margin-left: 20px;
                margin-top: 20px;
                font-size: 26px;
                position: relative;
                top: 100px;
            }
        </style>
        """,
        unsafe_allow_html=True,
    )

In [20]:
add_logo()

## Load Data

In [21]:
# Format for GitHub
df = pd.read_csv('data_storage/player_batting_season.csv')

# Format for Jupyter Notebook
#df = pd.read_csv('C:/Users/b7tbu/NINEBASE/ninebase/data_storage/player_batting_season.csv')

In [22]:
df.head()

,IDfg,Season,Name,Team,Age,G,AB,PA,H,1B,2B,3B,HR,R,RBI,BB,IBB,SO,HBP,SF,SH,GDP,SB,CS,AVG,GB,FB,LD,IFFB,Pitches,Balls,Strikes,IFH,BU,BUH,BB%,K%,BB/K,OBP,SLG,OPS,ISO,BABIP,GB/FB,LD%,GB%,FB%,IFFB%,HR/FB,IFH%,BUH%,wOBA,wRAA,wRC,Bat,Fld,Rep,Pos,RAR,WAR,Dol,Spd,wRC+,WPA,-WPA,+WPA,RE24,REW,pLI,phLI,PH,WPA/LI,Clutch,FB% (Pitch),FBv,SL%,SLv,CT%,CTv,CB%,CBv,CH%,CHv,SF%,SFv,KN%,KNv,XX%,PO%,wFB,wSL,wCT,wCB,wCH,wSF,wKN,wFB/C,wSL/C,wCT/C,wCB/C,wCH/C,wSF/C,wKN/C,O-Swing%,Z-Swing%,Swing%,O-Contact%,Z-Contact%,Contact%,Zone%,F-Strike%,SwStr%,BsR,FA% (sc),FT% (sc),FC% (sc),FS% (sc),FO% (sc),SI% (sc),SL% (sc),CU% (sc),KC% (sc),EP% (sc),CH% (sc),SC% (sc),KN% (sc),UN% (sc),vFA (sc),vFT (sc),vFC (sc),vFS (sc),vFO (sc),vSI (sc),vSL (sc),vCU (sc),vKC (sc),vEP (sc),vCH (sc),vSC (sc),vKN (sc),FA-X (sc),FT-X (sc),FC-X (sc),FS-X (sc),FO-X (sc),SI-X (sc),SL-X (sc),CU-X (sc),KC-X (sc),EP-X (sc),CH-X (sc),SC-X (sc),KN-X (sc),FA-Z (sc),FT-Z (sc),FC-Z (sc),FS-Z (sc),FO-Z (sc),SI-Z (sc),SL-Z (sc),CU-Z (sc),KC-Z (sc),EP-Z (sc),CH-Z (sc),SC-Z (sc),KN-Z (sc),wFA (sc),wFT (sc),wFC (sc),wFS (sc),wFO (sc),wSI (sc),wSL (sc),wCU (sc),wKC (sc),wEP (sc),wCH (sc),wSC (sc),wKN (sc),wFA/C (sc),wFT/C (sc),wFC/C (sc),wFS/C (sc),wFO/C (sc),wSI/C (sc),wSL/C (sc),wCU/C (sc),wKC/C (sc),wEP/C (sc),wCH/C (sc),wSC/C (sc),wKN/C (sc),O-Swing% (sc),Z-Swing% (sc),Swing% (sc),O-Contact% (sc),Z-Contact% (sc),Contact% (sc),Zone% (sc),Pace,Def,wSB,UBR,Age Rng,Off,Lg,wGDP,Pull%,Cent%,Oppo%,Soft%,Med%,Hard%,TTO%,CH% (pi),CS% (pi),CU% (pi),FA% (pi),FC% (pi),FS% (pi),KN% (pi),SB% (pi),SI% (pi),SL% (pi),XX% (pi),vCH (pi),vCS (pi),vCU (pi),vFA (pi),vFC (pi),vFS (pi),vKN (pi),vSB (pi),vSI (pi),vSL (pi),vXX (pi),CH-X (pi),CS-X (pi),CU-X (pi),FA-X (pi),FC-X (pi),FS-X (pi),KN-X (pi),SB-X (pi),SI-X (pi),SL-X (pi),XX-X (pi),CH-Z (pi),CS-Z (pi),CU-Z (pi),FA-Z (pi),FC-Z (pi),FS-Z (pi),KN-Z (pi),SB-Z (pi),SI-Z (pi),SL-Z (pi),XX-Z (pi),wCH (pi),wCS (pi),wCU (pi),wFA (pi),wFC (pi),wFS (pi),wKN (pi),wSB (pi),wSI (pi),wSL (pi),wXX (pi),wCH/C (pi),wCS/C (pi),wCU/C (pi),wFA/C (pi),wFC/C (pi),wFS/C (pi),wKN/C (pi),wSB/C (pi),wSI/C (pi),wSL/C (pi),wXX/C (pi),O-Swing% (pi),Z-Swing% (pi),Swing% (pi),O-Contact% (pi),Z-Contact% (pi),Contact% (pi),Zone% (pi),Pace (pi),FRM,AVG+,BB%+,K%+,OBP+,SLG+,ISO+,BABIP+,LD+%,GB%+,FB%+,HR/FB%+,Pull%+,Cent%+,Oppo%+,Soft%+,Med%+,Hard%+,EV,LA,Barrels,Barrel%,maxEV,HardHit,HardHit%,Events,CStr%,CSW%,xBA,xSLG,xwOBA,L-WAR
0,18401,2023,Ronald Acuna Jr.,ATL,25,86,347,396,117,71,24,1,21,78,54,43,2,49,4,2,0,7,41,7,0.33700,150,91,59,6,1528,617,911,16,0,0,0.10900,0.12400,0.88000,0.41400,0.59400,1.00800,0.25600,0.34400,0.01650,0.19700,0.50000,0.30300,0.06600,0.23100,0.10700,0.00000,0.42700,35.50000,83,33.50000,0.30000,12.20000,-3.90000,48.70000,4.90000,$39.0,6.80000,168,3.67000,-5.24000,8.91000,42.05000,4.33000,0.90000,NaN,0,3.65000,0.43000,0.50100,93.70000,0.23200,85.10000,0.06200,89.30000,0.08200,80.00000,0.09600,85.40000,0.02700,85.00000,NaN,NaN,0.00800,NaN,17.40000,11.30000,1.00000,6.40000,-0.90000,2.30000,NaN,2.28000,3.20000,1.09000,5.09000,-0.61000,5.73000,NaN,0.26700,0.72100,0.45600,0.72700,0.87100,0.82200,0.41600,0.58300,0.08100,5.40000,0.30400,NaN,0.06100,0.02300,0.00200,0.19100,0.23300,0.06100,0.02200,0.00300,0.09800,0.00100,NaN,NaN,94.00000,NaN,89.40000,84.70000,81.40000,94.10000,85.10000,79.10000,81.90000,54.20000,85.60000,81.30000,NaN,-2.30000,NaN,0.50000,-4.90000,-6.10000,-7.00000,1.90000,1.70000,3.30000,-3.90000,-1.60000,-7.00000,NaN,9.10000,NaN,5.00000,1.80000,-0.20000,5.20000,1.30000,-5.50000,-5.50000,6.90000,3.80000,-2.60000,NaN,12.90000,NaN,0.50000,1.90000,0.00000,4.00000,9.40000,5.50000,0.50000,0.40000,-0.40000,0.10000,NaN,2.78000,NaN,0.57000,5.45000,0.29000,1.37000,2.64000,5.94000,1.37000,10.70000,-0.30000,4.27000,NaN,0.23100,0.67800,0.45700,0.66100,0.87300,0.82000,0.50600,18.10000,-3.60000,4.40000,1.70000,25 - 25,38.90000,1.20000,-0.70000,0.45700,0.33300,0.21000,0.10000,0.46000,0.44000,0.28500,0.09900,NaN,0.08100,0.29700,0.06100,0.02700,NaN,0.00100,0.19800,0.23700,NaN,85.

In [31]:
import plotly.express as px
import statsmodels.api as sm

### Row 1 - Plot 1 (outdated)

fig_1 = px.scatter(
    df.query("Season==2023"),
    title="Hard%+ vs. BABIP+",
    x = "Hard%+",
    y = "BABIP+",
    hover_name = "Name",
    log_x = True,
    trendline = "ols",
    size_max = 60,
    height = 750,
    width = 750,
)

### Row 1 - Tab 1

In [59]:
fig_11 = px.box(
    df.query("Season==2023"),
    title="Hard Hit %+",
    y = "Hard%+",
    points="all",
    hover_name = "Name",
    height = 500,
    width = 500,
)

In [60]:
fig_12 = px.box(
    df.query("Season==2023"),
    title="Home Runs",
    y = "HR",
    points="all",
    hover_name = "Name",
    height = 500,
    width = 500,
)

In [61]:
fig_13 = px.box(
    df.query("Season==2023"),
    title="wRC+",
    y = "wRC+",
    points="all",
    hover_name = "Name",
    height = 500,
    width = 500,
)

### Row 1 - Tab 2

In [62]:
fig_21 = px.box(
    df.query("Season==2023"),
    title="Walk %+",
    y = "BB%+",
    points="all",
    hover_name = "Name",
    height = 500,
    width = 500,
)

In [71]:
fig_22 = px.box(
    df.query("Season==2023"),
    title="Strikeout %+",
    y = "K%+",
    points="all",
    hover_name = "Name",
    height = 500,
    width = 500,
)

In [68]:
fig_23 = px.box(
    df.query("Season==2023"),
    title="Out of Zone Swing %",
    y = "O-Swing%",
    points="all",
    hover_name = "Name",
    height = 500,
    width = 500,
)

### Row 1 - Print

In [72]:
# Subheader
st.subheader('Player Batting')

# Create the tabs
tabs = st.tabs(["Power", "Plate Discipline", "Clutch"])

# Display the charts within the tabs
with tabs[0]:
    col1, col2, col3 = st.columns(3)
    col1.plotly_chart(fig_11, theme="streamlit", use_container_width=False)
    col2.plotly_chart(fig_12, theme="streamlit", use_container_width=False)
    col3.plotly_chart(fig_13, theme="streamlit", use_container_width=False)

with tabs[1]:
    col4, col5, col6 = st.columns(3)
    col4.plotly_chart(fig_21, theme="streamlit", use_container_width=False)
    col5.plotly_chart(fig_22, theme="streamlit", use_container_width=False)
    col6.plotly_chart(fig_23, theme="streamlit", use_container_width=False)

## Row 2

### Row - Raw Data

In [74]:
from st_aggrid import AgGrid, GridOptionsBuilder, DataReturnMode, JsCode

In [75]:
# Simplify dataframe, narrow to most insightful columns
df_short = df[['Name', 'Team', 'Age', 'AB', 'BB%+', 'K%+', 'BABIP+', 'Hard%+', 'wRC+']]

In [24]:
# Builds a gridOptions dictionary using a GridOptionsBuilder instance.
builder = GridOptionsBuilder.from_dataframe(df_short)
builder.configure_side_bar(filters_panel=True, columns_panel=True)

# Columns

builder.configure_column("Name", header_name="Player", width=150, editable=False)
builder.configure_column("Team", width=100, enableRowGroup=True)
builder.configure_column("Age", width=100)
builder.configure_column("AB", width=100)
builder.configure_column("BB%+", width=100)
builder.configure_column('K%+', width=100)
builder.configure_column('BABIP+', width=100)
builder.configure_column('Hard%+', width=100)
builder.configure_column("wRC+", width=100, sort='desc')

# Launch
go = builder.build()

### Row 2 - Print

In [20]:
col1, col2, = st.columns([0.75, 0.25])

with col1:
    st.subheader("Raw Data")
    grid_response = AgGrid(
        df_short,
        gridOptions=go,
        theme="streamlit",
        height=600
    )
    
with col2:
    st.subheader("")

In [ ]:
st.divider()

### Playground

In [83]:
# Builds a gridOptions dictionary using a GridOptionsBuilder instance.
builder = GridOptionsBuilder.from_dataframe(df_short)
builder.configure_side_bar(filters_panel=True, columns_panel=True)

# Columns
builder.configure_column("Name", header_name="Player", width=150, editable=False)
builder.configure_column("Team", width=100, enableRowGroup=True)
builder.configure_column("Age", width=100)
builder.configure_column("AB", width=100)
builder.configure_column("BB%+", width=100)
builder.configure_column('K%+', width=100)
builder.configure_column('BABIP+', width=100)
builder.configure_column('Hard%+', width=100)
builder.configure_column("wRC+", width=100, sort='desc')

# Column Grouping
column_defs = [
    {
        "headerName": "Player Details",
        "children": [
            {"field": "Name", "headerName": "Player", pinned: 'left'},
            {"field": "Team"},
            {"field": "Age"},
            {"field": "AB", "headerName": "At Bats"},
        ]
    },
    {
        "headerName": "Power",
        "children": [
            {"field": "Hard%+"},
            {"field": "HR"},
            {"field": ""},
            {"field": "BABIP+"},
            {"field": "Hard%+"},
            {"field": "wRC+", "columnGroupShow": "open"},
        ]
    },
    {
        "headerName": "Plate Disciplline",
        "children": [
            {"field": "BB%+"},
            {"field": "K%+"},
        ]
    }
]

# Merge columnDefs with existing column definitions
grid_options = {"columnDefs": column_defs}

# Launch
go = grid_options

col1, col2 = st.columns([0.75, 0.25])

with col1:
    st.subheader("Raw Data")
    grid_response = AgGrid(
        df_short,
        gridOptions=go,
        theme="streamlit",
        height=600
    )

with col2:
    st.subheader("")